### 引入库

In [1]:
import pdfplumber
import pandas as pd
import numpy as np

### 扫描PDF，文本转存TXT
##### 预计同转存EXCEL模块合并为一个功能
##### 创建按钮时可以添加另存为功能？自己选择地址

In [2]:
##PDF文档存储地址
pdfPath = "./G120C_failure_code_table.pdf"

##依次扫描PDF页面并串联文本
pageText = ''
with pdfplumber.open(pdfPath) as pdf:
    for n in range(len(pdf.pages)):
        singlePage = pdf.pages[n]
        singlePageText = singlePage.extract_text()
        pageText = pageText + singlePageText

##完整文本转存TXT格式
txtPath = './G120C_Failure_Code_Table.txt'
with open(txtPath, 'w', encoding = 'utf-8') as c:
    c.write(pageText)
    c.close()

print("Done!")

Done!


### 从TXT提取关键信息并转存EXCEL（脑残版）
##### 作为一个小功能设额外按钮（虽然不一定用到）
##### 这一模块可直接作为一个py文件调用函数CreateExcel

In [4]:
txtPath = './G120C_Failure_Code_Table.txt'
## 获取每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 剔除不需要的信息行
with open(txtPath,"w",encoding="utf-8") as f_w:
    for line in allLine:
        if "故障和报警" in line:
            continue
        if "SINAMICS G120C" in line:
            continue
        if "参数手册," in line:
            continue
        f_w.write(line)

## 刷新每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 筛选分类文本信息
failure = {}                #故障码和名称
failureNumber = 0
failureLocation = {}

informationCatefory = {}        #信息类别
informationCateforyNumber = 0
informationCateforyLocation = {}

reason = {}
reasonNumber = 0                #原因数量
reasonLocation = {}           #原因所在首行

processing = {}
processingNumber = 0            #处理数量
processingLocation = {}       #处理所在首行

########################
###基于面向对象思想编写###
########################
class Foo:
    def __init__(self, search, name, number, lineNumber):
        self.search = search
        self.name = name
        self.number = number
        self.lineNumber = lineNumber

    def searchError(self):                              #检索故障码
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):    
            if self.search in allLine[x]:
                self.name[self.number] = allLine[x - 1]
                self.lineNumber[self.number] = x - 1
                self.number = self.number + 1 
                countNumber = self.number   

    def searchInformation(self):                        #检索信息值和信息类别
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):
            if self.search in allLine[x]:
                self.name[self.number] = allLine[x]
                self.lineNumber[self.number] = x
                self.number = self.number + 1
                countNumber = self.number
    
    def searchInformation1(self):                        #检索其他需求信息
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):
            if self.search in allLine[x]:
                self.lineNumber[self.number] = x
                self.number = self.number + 1
                countNumber = self.number

Failure = Foo('信息类别： ', failure, failureNumber, failureLocation)
Failure.searchError()
failureNumber = countNumber

InformationCatefory = Foo('信息类别： ', informationCatefory, informationCateforyNumber, informationCateforyLocation)
InformationCatefory.searchInformation()
informationCateforyNumber = countNumber

Reason = Foo('原因： ', reason, reasonNumber,reasonLocation)
Reason.searchInformation1()
reasonNumber = countNumber

Processing = Foo('处理： ', processing, processingNumber, processingLocation)
Processing.searchInformation1()
processingNumber = countNumber

##提取原因和处理
if reasonNumber != processingNumber:
    print("信息提取有误！")
    print(reasonNumber)
    print(processingNumber)
elif reasonNumber != failureNumber:
    print("信息提取有误！")
else:
    for x in range(reasonNumber):
        ##提取原因
        if reasonLocation[x] == processingLocation[x] - 1:          #如果原因只有一行
            reason[x] = allLine[reasonLocation[x]]
        else:
            dataReason = ''                                                           #如果原因有多行
            lineRange = processingLocation[x] - reasonLocation[x]
            for y in range(lineRange):
                lineNumber = reasonLocation[x] + y
                dataReason = dataReason + allLine[lineNumber]
                reason[x] = dataReason
        ##提取处理
        if x != processingNumber - 1:                                           #如果处理的不是最后一条
            if processingLocation[x] == failureLocation[x + 1] - 1:     #如果处理只有一行
                processing[x] = allLine[processingLocation[x]]
            else: 
                dataProcess = ''                                                              #如果处理有多行
                lineRange = failureLocation[x + 1] - processingLocation[x]
                for y in range(lineRange):
                    lineNumber = processingLocation[x] + y
                    dataProcess = dataProcess + allLine[lineNumber]
                    processing[x] = dataProcess
        else:
            dataProcess = ''
            lineRange = allLineNumber - processingLocation[x]
            for y in range(lineRange):   
                lineNumber = processingLocation[x] + y
                dataProcess = dataProcess + allLine[lineNumber]
                processing[x] = dataProcess

##将关键信息转存到EXCEL
excelPath = "./G120C_failure_code_table.xlsx"

def CreateExcel(information1, information2, information3, information4, path):
    data = {'故障名称':information1, '信息类别':information2, '原因':information3, '处理':information4}
    df = pd.DataFrame(data)
    df.to_excel(path) 

CreateExcel(failure, informationCatefory, reason, processing, excelPath)

print("Done!")

Done!


### 文本内容及其他信息提取（用于主程序）
##### 各关键词检索的文本内容、总数、位置均已记录

In [7]:
txtPath = './G120C_Failure_Code_Table.txt'
## 获取每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 剔除不需要的信息行
with open(txtPath,"w",encoding="utf-8") as f_w:
    for line in allLine:
        if "故障和报警" in line:
            continue
        if "SINAMICS S120/S150" in line:
            continue
        if "参数手册," in line:
            continue
        f_w.write(line)

## 刷新每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 筛选分类文本信息
failure = {}                #故障码和名称
failureNumber = 0
failureLocation = {}

informationCatefory = {}        #信息类别
informationCateforyNumber = 0
informationCateforyLocation = {}

reason = {}
reasonNumber = 0                #原因数量
reasonLocation = {}           #原因所在首行

processing = {}
processingNumber = 0            #处理数量
processingLocation = {}       #处理所在首行

#####################
###基于常规方法编写###
#####################
for x in range(allLineNumber):
    if "信息类别： " in allLine[x]:
        # print(allLine[i])
        failure[failureNumber] = allLine[x - 1]                 #故障码和名称
        failureLocation[failureNumber] = x - 1                  #故障码所在行
        informationCatefory[failureNumber] = allLine[x]         #信息类别
        failureNumber = failureNumber + 1
        informationCateforyNumber = informationCateforyNumber + 1
    if "原因： " in  allLine[x]:
        reasonLocation[reasonNumber] = x
        reasonNumber = reasonNumber + 1
    if "处理： " in allLine[x]:
        processingLocation[processingNumber] = x
        processingNumber = processingNumber + 1

##提取原因和处理
if reasonNumber != processingNumber:
    print("信息提取有误！")
    print(reasonNumber)
    print(processingNumber)
elif reasonNumber != failureNumber:
    print("信息提取有误！")
else:
    for x in range(reasonNumber):
        ##提取原因
        if reasonLocation[x] == processingLocation[x] - 1:              #如果原因只有一行
            reason[x] = allLine[reasonLocation[x]]
        else:
            dataReason = ''                                             #如果原因有多行
            lineRange = processingLocation[x] - reasonLocation[x]
            for y in range(lineRange):
                lineNumber = reasonLocation[x] + y
                dataReason = dataReason + allLine[lineNumber]
                reason[x] = dataReason
        ##提取处理
        if x != processingNumber - 1:                                   #如果处理的不是最后一条
            if processingLocation[x] == failureLocation[x + 1] - 1:     #如果处理只有一行
                processing[x] = allLine[processingLocation[x]]
            else: 
                dataProcess = ''                                        #如果处理有多行
                lineRange = failureLocation[x + 1] - processingLocation[x]
                for y in range(lineRange):
                    lineNumber = processingLocation[x] + y
                    dataProcess = dataProcess + allLine[lineNumber]
                    processing[x] = dataProcess
        else:
            dataProcess = ''
            lineRange = allLineNumber - processingLocation[x]
            for y in range(lineRange):   
                lineNumber = processingLocation[x] + y
                dataProcess = dataProcess + allLine[lineNumber]
                processing[x] = dataProcess

'''切割故障码和故障名称'''
###输入存有故障码和名称的词典failure以及故障码总数failureNumber
###输出分别存有故障码和故障名的两个词典code和name
def cutFailure(information, number):
    code = {}
    name = {}

    for n in range(number):
        str = information[n]
        str2 = str.split(' ', 1)
        code[n] = str2[0]
        name[n] = str2[1]

    return code, name

failureCode, failureName = cutFailure(failure, failureNumber)

########################
##提取指定故障码的相关信息（提取单个故障码）
########按钮功能#########
########################
###输入故障码词典、故障码数量、故障码所在行位置词典、要检索的故障码
###输出被检索的故障码和相关信息
def GetFailureInformation(codeList, number, location, targetCode):
    missionComplete = False     #故障码查询结果标志位
    if len(targetCode) != 6:
        missionComplete = False
    else:   
        for m in range(number):
            if targetCode in codeList[m]:
                targetNumber = m
                missionComplete = True           

    if missionComplete == True:    
        targetLocationUp = location[targetNumber]
        targetLocationDown = location[targetNumber + 1]
        targetRange = targetLocationDown - targetLocationUp

        dataTarget = ''
        for n in range(targetRange):
            lineNumber = targetLocationUp + n
            dataTarget = dataTarget + allLine[lineNumber]
        return dataTarget
    else:
        missionFailed = '您输入的故障码有误，请核验后再次输入！\n'
        return missionFailed        

searchingCode = 'F01023'
GetFailureInformation(failureCode, failureNumber, failureLocation, searchingCode)

'F01023 内部软件超时\n信息类别： 硬件/软件故障 (1)\n反应： 无\n应答： 立即\n原因： 出现了内部软件超时。\n故障值（r0949，十进制）：\n仅用于西门子内部的故障诊断。\n处理： -重新为所有组件上电（断电/上电）。\n-将固件升级到新版本。\n- 联系技术支持。\n'